In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import gc
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn import metrics

plt.style.use('seaborn')
sns.set(font_scale=2)
pd.set_option('display.max_columns', 500)

# 1. Read and check dataset

## 1.1 Read dataset

- This parted was taken from the helpful kernel. https://www.kaggle.com/theoviel/load-the-totality-of-the-data

In [ ]:
!pip install scipy==1.2.0


In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [ ]:
%time train = pd.read_csv("../input/train.csv", dtype=dtypes)
%time test = pd.read_csv("../input/test.csv", dtype=dtypes)

In [ ]:
print(train.shape, test.shape)

- You can see that the datasets are large.

## 1.2 Check the target

In [ ]:
train['HasDetections'].value_counts().plot.bar()
plt.title('HasDetections(target)')

- Wow, very-well balanced target! Fun with this competition :).

## 1.2 Check the dataset

In [ ]:
%%time
# checking missing data
total = train.isnull().sum().sort_values(ascending = False)
percent = (train.isnull().sum()/train.isnull().count()*100).sort_values(ascending = False)
missing_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])


In [ ]:
missing_train_data.head(50)

- PuaMode, Census_ProcessorClass, DefaultBrowsersIdentifier, Census_IsFlightingInternal and Census_InternalBatteryType have over 70% null data.
- Let's check their distribution regarding to the target.
- Because datasets are large, let's compare the distributions using 10% of train.[](http://)

In [ ]:
train_small = train.sample(frac=0.1).copy()

### 1.1.1 PuaMode

In [ ]:
print(train_small['PuaMode'].dtypes)

In [ ]:
sns.countplot(x='PuaMode', hue='HasDetections',data=train_small)
plt.show()

- Some difference exists there. But, the samples are quite few, so remove this feature.

### 1.1.2 Census_ProcessorClass

In [ ]:
print(train_small['Census_ProcessorClass'].dtypes)

In [ ]:
sns.countplot(x='Census_ProcessorClass', hue='HasDetections',data=train_small)
plt.show()

- The meaning of 'Census_ProcessorClassr' is 'Number of logical cores in the processor'.
- You can check that the more logical cores, the more probable infection with malwares.
- This feature could be a good features only or component for the combinations with other features. Keep this and think it!

### 1.1.3 DefaultBrosersIdentifier

In [ ]:
print(train_small['DefaultBrowsersIdentifier'].dtypes)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, 'DefaultBrowsersIdentifier'], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, 'DefaultBrowsersIdentifier'], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, 'DefaultBrowsersIdentifier'].hist(ax=ax[1])
train_small.loc[train['HasDetections'] == 1, 'DefaultBrowsersIdentifier'].hist(ax=ax[1])
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])

plt.show()

- DefaultBrowsersIdentifier means ' ID for the machine's default browser'.
- Is this feature meaningful?

### 1.1.4 Census_IsFightingInternal

In [ ]:
print(train_small['Census_IsFlightingInternal'].dtypes)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, 'Census_IsFlightingInternal'], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, 'Census_IsFlightingInternal'], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, 'Census_IsFlightingInternal'].hist(ax=ax[1])
train_small.loc[train['HasDetections'] == 1, 'Census_IsFlightingInternal'].hist(ax=ax[1])
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])

plt.show()

In [ ]:
train_small.loc[train['HasDetections'] == 1, 'Census_IsFlightingInternal'].value_counts()

In [ ]:
train_small.loc[train['HasDetections'] == 0, 'Census_IsFlightingInternal'].value_counts()

- As you can see, The two samples have 1.0 value for Census_IsFlightingInternal. Just remove.

### 1.1.5 Census_InternalBatteryType

In [ ]:
print(train_small['Census_InternalBatteryType'].dtypes)

In [ ]:
train_small['Census_InternalBatteryType'].value_counts()

- I think this feature means the type of batteries of each machine.
- Oh, no....These days, most batteries are lithum-ion battery.
- So, Let's group them into lithum-batter group and non0-lithum-battery group

In [ ]:
def group_battery(x):
    x = x.lower()
    if 'li' in x:
        return 1
    else:
        return 0
    
train_small['Census_InternalBatteryType'] = train_small['Census_InternalBatteryType'].apply(group_battery)

In [ ]:
sns.countplot(x='Census_InternalBatteryType', hue='HasDetections',data=train_small)
plt.show()

- The difference is quite small. Do you think that some malwares recognize and select machine based on the type of battery?
- Battery is very important part for life of machine. I think that malware will focus on other hardware and software parts of machine. remove this.

In [ ]:
null_cols_to_remove = ['DefaultBrowsersIdentifier', 'PuaMode',
                       'Census_IsFlightingInternal', 'Census_InternalBatteryType']

train.drop(null_cols_to_remove, axis=1, inplace=True)
test.drop(null_cols_to_remove, axis=1, inplace=True)

## 2. Exploratory data analysis

- Exploring data is an exciting journey. During this, we can mine some interesting intuition for high AUC!

## 2.1 Categorical features

In [ ]:
categorical_features = [
        'ProductName',                                          
        'EngineVersion',                                        
        'AppVersion',                                           
        'AvSigVersion',                                         
        'Platform',                                             
        'Processor',                                            
        'OsVer',                                                
        'OsPlatformSubRelease',                                 
        'OsBuildLab',                                           
        'SkuEdition',                                           
        'SmartScreen',                                          
        'Census_MDC2FormFactor',                                
        'Census_DeviceFamily',                                  
        'Census_PrimaryDiskTypeName',                           
        'Census_ChassisTypeName',                               
        'Census_PowerPlatformRoleName',                         
        'Census_OSVersion',                                     
        'Census_OSArchitecture',                                
        'Census_OSBranch',                                      
        'Census_OSEdition',                                     
        'Census_OSSkuName',                                     
        'Census_OSInstallTypeName',                             
        'Census_OSWUAutoUpdateOptionsName',                     
        'Census_GenuineStateName',                              
        'Census_ActivationChannel',                             
        'Census_FlightRing',                                    
]

### 2.1.1 ProductName - Defender state information e.g. win8defender

In [ ]:
def plot_category_percent_of_target(col):
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    cat_percent = train_small[[col, 'HasDetections']].groupby(col, as_index=False).mean()
    cat_size = train_small[col].value_counts().reset_index(drop=False)
    cat_size.columns = [col, 'count']
    cat_percent = cat_percent.merge(cat_size, on=col, how='left')
    cat_percent['HasDetections'] = cat_percent['HasDetections'].fillna(0)
    cat_percent = cat_percent.sort_values(by='count', ascending=False)[:20]
    sns.barplot(ax=ax, x='HasDetections', y=col, data=cat_percent, order=cat_percent[col])

    for i, p in enumerate(ax.patches):
        ax.annotate('{}'.format(cat_percent['count'].values[i]), (p.get_width(), p.get_y()+0.5), fontsize=20)

    plt.xlabel('% of HasDetections(target)')
    plt.ylabel(col)
    plt.show()

In [ ]:
col = categorical_features[0]
plot_category_percent_of_target(col)

### 2.1.2 EngineVersion - Defender state information e.g. 1.1.12603.0

In [ ]:
col = categorical_features[1]
plot_category_percent_of_target(col)

### 2.1.3 AppVersion - Defender state information e.g. 4.9.10586.0

In [ ]:
col = categorical_features[2]
plot_category_percent_of_target(col)

### 2.1.4 AvSigVersion - Defender state information e.g. 1.217.1014.0

In [ ]:
col = categorical_features[3]
plot_category_percent_of_target(col)

### 2.1.5 Platform - Calculates platform name (of OS related properties and processor property)

In [ ]:
col = categorical_features[4]
plot_category_percent_of_target(col)

### 2.1.6 Processor - This is the process architecture of the installed operating system

In [ ]:
col = categorical_features[5]
plot_category_percent_of_target(col)

### 2.1.7 Census_OSVersion - Numeric OS version Example - 10.0.10130.0

In [ ]:
col = categorical_features[6]
plot_category_percent_of_target(col)

### 2.1.8 OsPlatformSubRelease - Returns the OS Platform sub-release (Windows Vista, Windows 7, Windows 8, TH1, TH2)

In [ ]:
col = categorical_features[7]
plot_category_percent_of_target(col)

### 2.1.9 OsBuildLab - Build lab that generated the current OS. Example: 9600.17630.amd64fre.winblue_r7.150109-2022

In [ ]:
col = categorical_features[8]
plot_category_percent_of_target(col)

### 2.1.10 SkuEdition - The goal of this feature is to use the Product Type defined in the MSDN to map to a 'SKU-Edition' name that is useful in population reporting. 

In [ ]:
col = categorical_features[9]
plot_category_percent_of_target(col)

### 2.1.11 SmartScreen - This is the SmartScreen enabled string value from registry. This is obtained by checking in order, 

In [ ]:
col = categorical_features[10]
plot_category_percent_of_target(col)

### 2.1.12 Census_MDC2FormFactor - A grouping based on a combination of Device Census level hardware characteristics.**

In [ ]:
col = categorical_features[11]
plot_category_percent_of_target(col)

### 2.1.13 Census_DeviceFamily - AKA DeviceClass. Indicates the type of device that an edition of the OS is intended for. 

In [ ]:
col = categorical_features[12]
plot_category_percent_of_target(col)

### 2.1.14 Census_PrimaryDiskTotalCapacity - Amount of disk space on primary disk of the machine in MB

In [ ]:
col = categorical_features[13]
plot_category_percent_of_target(col)

### 2.1.15 Census_ChassisTypeName - Retrieves a numeric representation of what type of chassis the machine has

In [ ]:
col = categorical_features[14]
plot_category_percent_of_target(col)

### 2.1.16 Census_PowerPlatformRoleName - Indicates the OEM preferred power management profile

In [ ]:
col = categorical_features[15]
plot_category_percent_of_target(col)

### 2.1.17 Census_OSVersion - Numeric OS version Example

In [ ]:
col = categorical_features[16]
plot_category_percent_of_target(col)

### 2.1.18 Census_OSArchitecture - Architecture on which the OS is based

In [ ]:
col = categorical_features[17]
plot_category_percent_of_target(col)

### 2.1.19 Branch of the OS extracted from the OsVersionFull

In [ ]:
col = categorical_features[18]
plot_category_percent_of_target(col)

### 2.1.20 Census_OSEdition - Edition of the current OS

In [ ]:
col = categorical_features[19]
plot_category_percent_of_target(col)

### 2.1.21 Census_OSSkuName - OS edition friendly name

In [ ]:
col = categorical_features[20]
plot_category_percent_of_target(col)

### 2.1.22 Census_PrimaryDiskTypeName - Friendly name of Primary Disk Type - HDD or SSD

In [ ]:
col = categorical_features[21]
plot_category_percent_of_target(col)

### 2.1.23 Census_OSWUAutoUpdateOptionsName - Friendly name of the WindowsUpdate auto-update settings on the machine.

In [ ]:
col = categorical_features[22]
plot_category_percent_of_target(col)

### 2.1.24 Census_GenuineStateName - Friendly name of OSGenuineStateID. 0 = Genuine

In [ ]:
col = categorical_features[23]
plot_category_percent_of_target(col)

### 2.1.25 Census_ActivationChannel - Retail license key or Volume license key for a machine.

In [ ]:
col = categorical_features[24]
plot_category_percent_of_target(col)

### 2.1.26 Census_IsFlightingInternal - NA

In [ ]:
col = categorical_features[25]
plot_category_percent_of_target(col)

## 2.2 Float features

In [ ]:
float_features = [
        'RtpStateBitfield',                                     
        'DefaultBrowsersIdentifier',                            
        'AVProductStatesIdentifier',                            
        'AVProductsInstalled',                                  
        'AVProductsEnabled',                                    
        'CityIdentifier',                                       
        'OrganizationIdentifier',                               
        'GeoNameIdentifier',                                    
        'IsProtected',                                          
        'SMode',                                                
        'IeVerIdentifier',                                      
        'Firewall',                                             
        'UacLuaenable',                                         
        'Census_OEMNameIdentifier',                             
        'Census_OEMModelIdentifier',                            
        'Census_ProcessorCoreCount',                            
        'Census_ProcessorManufacturerIdentifier',               
        'Census_ProcessorModelIdentifier',                      
        'Census_PrimaryDiskTotalCapacity',                      
        'Census_SystemVolumeTotalCapacity',                     
        'Census_TotalPhysicalRAM',                              
        'Census_InternalPrimaryDiagonalDisplaySizeInInches',    
        'Census_InternalPrimaryDisplayResolutionHorizontal',    
        'Census_InternalPrimaryDisplayResolutionVertical',      
        'Census_InternalBatteryNumberOfCharges',                
        'Census_OSInstallLanguageIdentifier',                   
        'Census_IsFlightingInternal',                           
        'Census_IsFlightsDisabled',                             
        'Census_ThresholdOptIn',                                
        'Census_FirmwareManufacturerIdentifier',                
        'Census_FirmwareVersionIdentifier',                     
        'Census_IsWIMBootEnabled',                              
        'Census_IsVirtualDevice',                               
        'Census_IsAlwaysOnAlwaysConnectedCapable',              
        'Wdft_IsGamer',                                         
        'Wdft_RegionIdentifier',                                
]

### 2.2.1 RtpStateBitfield - NA

In [ ]:
col = float_features[0]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[1]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[2]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[3]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[4]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[5]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[6]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[7]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[8]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[9]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[10]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[11]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[12]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[13]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

In [ ]:
col = float_features[14]
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

plt.suptitle(col, fontsize=30)
ax[0].set_yscale('log')
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
ax[1].set_yscale('log')
plt.show()

# To be updated. If you think this is helpful, please upvote!